In [1]:
1

1

In [2]:
import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score


In [3]:
def get_message(mes):
    dic = {}
    dic['message'] = mes.text.split('\n\n')[-2]
    dic['message_id'] = mes.get('id').replace('message', '')
    dic['from_name'] = mes.find(attrs={'class':'from_name'})
    reply = mes.find(attrs = {'class': 'reply_to details'})
    
    # print(reply)
    if reply:
        dic['go_to_message_id'] = reply.find('a').get('href').replace('#go_to_message', '')
    return dic

In [4]:
# папки с ChatExport
sub_list = []
folders = glob.glob("C:/Users/artem/Downloads/Telegram Desktop/ChatExport/ChatExport*")
files = []
for folder in folders[:5]:
    files = glob.glob(f"{folder}/*.html")
    messages_all = []
    for file in tqdm(files):
        with open(file, encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            messages = soup.body.find_all(attrs = {'class':'message'})#.text#.split('\n\n')[-2].lower()
        messages_all+=messages
    chat_name = soup.find(attrs = {'class', "text bold"}).text
    df_sub = pd.DataFrame(list(map(get_message, messages_all)))
    df_sub["chat_name"] = chat_name
    sub_list.append(df_sub)
df = pd.concat(sub_list)

100%|██████████| 180/180 [01:48<00:00,  1.66it/s]


In [28]:
# df

In [5]:
df['message'] = df['message'].apply(lambda x: x.lower())

In [6]:
df = df.drop_duplicates()

target = 'question'
target_list = []
for message in df['message'].values:
    if 'посовет' in message:
        target_list.append(1)
    else:
        target_list.append(0)   

df[target] = target_list

In [7]:
df_reply_mapping = df[~df['go_to_message_id'].isnull()]
df_reply_mapping = df_reply_mapping.rename(columns={"go_to_message_id":"message_id", "message_id": "reply_id"})
df_reply_mapping = pd.merge(df, df_reply_mapping, on=['message_id', 'chat_name'], suffixes=['_question', '_reply'])

In [8]:
df_reply_mapping = df_reply_mapping[df_reply_mapping['question_question']==1]

In [9]:
# df_train, df_test = train_test_split(df, test_size=0.33,random_state=42)

# count_vector = CountVectorizer()
# tfidf_transformer = TfidfTransformer()

# X_train_counts = count_vector.fit_transform(df_train['message'])
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# # TF-IDF vectorize.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# # Create model(naive bayes) and training. 
# clf = MultinomialNB().fit(X_train_tfidf, df_train['question'])

# # Create test documents and vectorize.
# docs_new = df_test['message'].values
# X_new_counts = count_vector.transform(docs_new)
# X_new_tfidf = tfidf_transformer.transform(X_new_counts)

# # Execute prediction(classification).
# predicted = clf.predict_log_proba(X_new_tfidf)

# df_test['log_proba'] = predicted[:,1]

# df_test[df_test['question']==0].sort_values('log_proba').iloc[-50:]

# roc_auc_score(df_test[target], df_test['log_proba'])

In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
pd.options.display.width = 300

In [12]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [13]:
def tokenize(text):
    stop_words = ['за', 'там', 'так', 'то', 'все', 'если', '-',
       ':', 'но', 'как', 'по', 'у', 'есть', 'это', 'я', 'с', 'а', 'не', ')', 'и', '.', 'в', ',']
    tokens = word_tokenize(text, language="russian")
    tokens = list(filter(lambda x: x not in stop_words, tokens))
    
    return list(set(tokens))

In [14]:
df_reply_mapping['message_question_tokens'] = df_reply_mapping['message_question'].apply(tokenize)

In [15]:
df_questions = df[df['question']==1]
df_questions = df_questions[['message','message_id', 'chat_name', 'from_name']]

In [16]:
df_questions['message_tokens'] = df_questions['message'].apply(tokenize)

In [17]:
df_questions.head(2)

,message,message_id,chat_name,from_name,message_tokens
564,"у меня есть текущая работа кем то вроде эникея - сисадмина на частной вилле, работа не пыльная, сменная и достаточно скучная. на объекте есть оборудование - как бы умный дом, автоматизация, видеонаблюдение и все такое. все бы хорошо если бы не зп. в момент мне стало скучно и я вспомнил некогда горячо любимую java, полез в spring mvc, vue.js в общем обновлять знания и т.д. с конкретной целью - в поисках лучшей жизни устроиться куда- нибудь джуном: есть куда рости -раз, рост знаний и опыта пропорционален зп- два (поправьте, коль не так). сижу себе учусь по немногу, тестовые всякие делаю. тут текущий работодатель это дело видя предлагает поучить simpl control system programming software и соответственно зп увеличить по итогу. и вот собственно я бы хотел посоветоваться как поступить, понимаю что многие меня знать не знают, но тем не менее, может слышали что то про это чудо (simpl). просто в разработку давно хотелось, но пошел работать сюда и сижу с постоянным чувством что меня нае....ли. тут мне опять что то предлагают и вроде вкусно, а то что интересно оно вроде уже и почти близко то было... в общем хочу узнать ваши мнения.\n",660,\nIT Сочи Кофейня \n,None,"[хочу, умный, скучно, близко, интересно, пошел, мне, общем, хорошо, хотел, давно, стало, лучшей, предлагает, работа, работать, опять, spring, текущая, рост, посоветоваться, нибудь, зп, сисадмина, автоматизация, итогу, почти, ваши, горячо, коль, жизни, чудо, обновлять, делаю, разработку, мнения, вкусно, джуном, видя, ли, некогда, собственно, учусь, работодатель, пыльная, любимую, что, знают, тестовые, всякие, system, software, себе, сменная, оно, simpl, знания, частной, -раз, сюда, поправьте, ..., т.д., опыта, соответственно, на, устроиться, момент, поучить, mvc, дом, такое, хотелось, целью, поисках, ...., было, многие, тем, увеличить, понимаю, куда-, знаний, нае, два, знать, может, узнать, слышали, менее, эникея, зп-, (, java, просто, достаточно, вилле, про, полез, меня, ...]"
1648,"для меня нет авторитетов и кумиров, даже родители, я уже лет в 18 понял что люди которые жили в ссср адекватно не смогут посоветовать или понять как надо жить в настоящем, а тем более знать как что в будущем будет\n",11344,\nIT Сочи Кофейня \n,None,"[жить, 18, смогут, понять, или, ссср, для, меня, жили, более, уже, которые, кумиров, даже, что, тем, будет, понял, адекватно, надо, авторитетов, знать, родители, настоящем, лет, будущем, посоветовать, люди, нет]"


In [18]:
df_reply_mapping['message_tokens'] = df_reply_mapping['message_question'].apply(tokenize)

In [19]:
df_reply_mapping['key'] = 0
df_questions['key'] = 0

In [56]:
def intersect(row):
    stop_words = {'посоветуйте', 'пожалуйста', 'хорошего', 'хорошую', 'ребята', 'привет',
                  'здравствуйте', 'добрый', 'день', 'кто', 'может', 'посоветуйте', 'кто', 'может',
                 'хорошего', 'мастера',
                  'кто', 'привет', '!', '?', 'посоветовать', 'всем'
                  , 'парикмахерскую', 'может', 'хорошую', 'можете', 'добрый',
                'хорошего', 
                  'день', 'добрый', '!', 'посоветуйте', 'парикмахера',
                 }
    message_question_tokens = set(row['message_question_tokens']) - stop_words
    message_tokens = set(row['message_tokens']) - stop_words
    len1 = len(message_question_tokens&message_tokens)
    len2 = max(len(message_question_tokens), len(message_tokens))
    if len2==0:
        return 0
    return len1/len2

In [57]:
# df_reply_mapping

In [58]:
df_cross = pd.merge(
    df_reply_mapping[['message_question', 'message_id', 'message_question_tokens', 'key', 'from_name_question', 'from_name_reply']],
    df_questions,
    on='key')

df_cross = df_cross[(df_cross['message_id_x']!=df_cross['message_id_y'])&(df_cross['message_question']!=df_cross['message'])]
df_cross['intersect'] = df_cross.apply(intersect,axis=1)
df_cross = df_cross[df_cross['message_question']!=df_cross['message']]
df_cross = df_cross.sort_values('intersect')

In [59]:
pd.options.display.max_rows = 100

In [60]:
view = df_cross[(df_cross['intersect'].notnull())&(df_cross['from_name_question']!=df_cross['from_name'])].iloc[-140:-70]

,message_question,message_id_x,message_question_tokens,key,from_name_question,from_name_reply,message,message_id_y,chat_name,from_name,message_tokens,intersect
461454,"посоветуйте, пожалуйста мастера по маникюру 💅\n",35408,"[посоветуйте, мастера, 💅, пожалуйста, маникюру]",0,[\nЕлена Верстакова \n ],None,добрый день!!! девочки посоветуйте пожалуйста мастера по маникюру!\n,31457,\n🌄 Жители Красной Поляны \n,[\nLёka \n ],"[день, добрый, !, посоветуйте, мастера, пожалуйста, маникюру, девочки]",0.500000
864119,посоветуйте пожалуйста фотографа на поляне 🙏🏼\n,174481,"[фотографа, на, 🙏🏼, посоветуйте, пожалуйста, поляне]",0,[\nValentina Valentina\n ],[\nMatthew \n ],"всем привет! есть на поляне мастер по ребалансингу? посоветуйте , пожалуйста!\n",290420,\n💬 Красная Поляна чат \n,[\nAnahita A\n ],"[ребалансингу, на, привет, !, ?, посоветуйте, всем, пожалуйста, поляне, мастер]",0.500000
998638,всем привет. посоветуйте хорошего массажиста на красной поляне пожалуйста\n,196175,"[хорошего, массажиста, красной, на, привет, посоветуйте, всем, пожалуйста, поляне]",0,[\nВалентина \n ],[\n. \n ],"всем привет! есть на поляне мастер по ребалансингу? посоветуйте , пожалуйста!\n",290420,\n💬 Красная Поляна чат \n,[\nAnahita A\n ],"[ребалансингу, на, привет, !, ?, посоветуйте, всем, пожалуйста, поляне, мастер]",0.500000
2768640,всем привет! посоветуйте пожалуйста хорошего массажиста в поляне\n,149154,"[хорошего, массажиста, привет, !, посоветуйте, всем, пожалуйста, поляне]",0,[\nКуница \n ],[\nЕвгения Кудрина (Синичкина)\n ],посоветуйте костоправа в поляне ?\n,109309,\n💬 Красная Поляна чат \n,[\nСергей Иванов\n ],"[поляне, посоветуйте, костоправа, ?]",0.500000
586832,"а посоветуйте фотографа в поляне, пожалуйста\n",52741,"[посоветуйте, фотографа, поляне, пожалуйста]",0,[\nSvetlana E\n ],[\nАндрей Белянцев\n ],"посоветуйте хорошего колориста в поляне, пожалуйста )\n",141972,\n💬 Красная Поляна чат \n,[\nЮлия Серкина\n ],"[хорошего, колориста, посоветуйте, пожалуйста, поляне]",0.500000
1240791,"ребят, а посоветуйте в эсто-садке где вкусно поесть и выпить\n",222455,"[ребят, вкусно, выпить, посоветуйте, поесть, эсто-садке, где]",0,[\nEvgeniya Belskaya\n ],[\nLinar \n ],"добрый вечер! посоветуйте, пожалуйста, где вкусно поесть в красной поляне.\n",78434,\n🌄 Жители Красной Поляны \n,[\nMeow \n ],"[вкусно, добрый, красной, !, посоветуйте, вечер, поесть, пожалуйста, где, поляне]",0.500000
583886,"а посоветуйте фотографа в поляне, пожалуйста\n",52741,"[посоветуйте, фотографа, поляне, пожалуйста]",0,[\nSvetlana E\n ],[\nTatiana Hush\n ],"посоветуйте, пожалуйста, стоматологию в поляне\n",68370,\n🌄 Жители Красной Поляны \n,[\nelena \n ],"[стоматологию, посоветуйте, пожалуйста, поляне]",0.500000
2791557,здравствуйте!посоветуйте парикмахерскую в поляне .\n,152873,"[здравствуйте, !, посоветуйте, парикмахерскую, поляне]",0,[\nSerj \n ],[\nSergei Meldougaz Wandergazx\n ],"всем привет, посоветуйте сварщика в поляне.\n",143307,\n💬 Красная Поляна чат \n,None,"[привет, посоветуйте, всем, сварщика, поляне]",0.500000
694164,"добрый день , посоветуйте хорошего электрика , спасибо\n",73124,"[хорошего, день, добрый, спасибо, посоветуйте, электрика]",0,[\nDmitriy Zhuravel\n ],[\nDinara Malinovskaya\n ],"всем привет!посоветуйте, пож-та, электрика.\n",197235,\n💬 Красная Поляна чат \n,[\nJulia \n ],"[привет, !, посоветуйте, всем, электрика, пож-та]",0.500000
1916986,"всем, привет! посоветуйте хорошего барбера в поляне\n",290419,"[хорошего, привет, !, посоветуйте, всем, поляне, барбера]",0,[\nMichael \n ],[\nMarta \n ],"добрый день, посоветуйте пожалуйста грумера хорошего в поляне\n",178513,\n💬 Красная Поляна чат \n,[\nDeleted Account\n ],"[хорошего, день, добрый, грумера, посоветуйте, пожалуйста, поляне]",0.500000


In [105]:
new_questing = 'посоветуйте автошколу в сочи'

In [106]:
new_tokens = tokenize(new_questing)

In [107]:
new_tokens

['посоветуйте', 'автошколу', 'сочи']

In [108]:
df_reply_mapping['message_tokens'] = len(df_reply_mapping)*[new_tokens]

In [109]:
# df_reply_mapping

In [110]:
df_reply_mapping['score'] = df_reply_mapping.apply(intersect,axis=1)

In [111]:
df_reply_mapping.sort_values('score').iloc[-10:][['message_question', 'message_reply','message_question_tokens', 'score']]

,message_question,message_reply,message_question_tokens,score
115880,\nмогу посоветовать в сочи хорошего стоматолога\n,"\nспасибо, мне бы поближе\n","[хорошего, сочи, могу, стоматолога, посоветовать]",0.333333
81234,кто-нибудь может посоветовать хорошего дерматолога в сочи?!\n,"\nмухамедзянова илюса габдуллиевна в адлере, медцентр «даниэль» понравилась.\n","[хорошего, кто-нибудь, дерматолога, сочи, !, ?, посоветовать, может]",0.333333
52904,здравствуйте посоветуйте автошколу пожалуйста\n,"\nюжная, адлер\n","[посоветуйте, автошколу, пожалуйста, здравствуйте]",0.500000
2019,"всем привет, извиняюсь за оффтоп, но может кто может посоветовать автошколу в сочи?\n","\nя учился в автошколе при гаи, на пластунке. школа нормальная, правильная итд :)ездить научили, сдал нормально без всяких там.\n","[оффтоп, автошколу, привет, кто, сочи, ?, посоветовать, всем, может, извиняюсь]",0.500000
2018,"всем привет, извиняюсь за оффтоп, но может кто может посоветовать автошколу в сочи?\n",\nезди на такси и не трать свое время лишними заботами))\n,"[оффтоп, автошколу, привет, кто, сочи, ?, посоветовать, всем, может, извиняюсь]",0.500000
2017,"всем привет, извиняюсь за оффтоп, но может кто может посоветовать автошколу в сочи?\n",\nв кофейне оффтоп - это про ит в сочи )\n,"[оффтоп, автошколу, привет, кто, сочи, ?, посоветовать, всем, может, извиняюсь]",0.500000
2016,"всем привет, извиняюсь за оффтоп, но может кто может посоветовать автошколу в сочи?\n",\nраз уж пошел оффтоп присоединяюсь к вопросу\n,"[оффтоп, автошколу, привет, кто, сочи, ?, посоветовать, всем, может, извиняюсь]",0.500000
85235,посоветуйте хорошего фотографа в сочи\n,"\nденис вялов, очень крутой фотограф!!\n","[хорошего, посоветуйте, фотографа, сочи]",0.500000
85234,посоветуйте хорошего фотографа в сочи\n,\nкакой вид съёмки?)\n,"[хорошего, посоветуйте, фотографа, сочи]",0.500000
85236,посоветуйте хорошего фотографа в сочи\n,\n@nosov_art профессионал!\n,"[хорошего, посоветуйте, фотографа, сочи]",0.500000


In [ ]:
df['tokens'] = df['text'].apply(tokenize)

In [ ]:
df = df[df['tokens'].apply(len)>=3]

In [ ]:
pd.options.display.max_colwidth = 200

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
1

In [ ]:
df_train[df_train['question']==1].sample(n=30)

In [ ]:
train_lenght = len(df_train)

In [ ]:
mean_target = df_train['question'].mean()

In [ ]:
from functools import reduce

In [ ]:
from collections import Counter

In [ ]:
for text in df[df['question']==1]['text'].values:
    pass
    # print(text)
    # print()

In [ ]:
# df['question'].sum()

In [ ]:
from tqdm import tqdm

In [ ]:
total_count = Counter()
for tokens in tqdm(df_train['tokens'].values):
    for token in tokens:
        total_count[token]+=1
    

In [ ]:
# total_count

In [ ]:
target_count = Counter()
for tokens in tqdm( df_train[df_train[target]==1]['tokens'].values):
    for token in tokens:
        target_count[token]+=1

In [ ]:
pd.Series(total_count).sort_values().index[-30:]

In [ ]:
pd.Series(target_count).sort_values()[-30:]

In [ ]:
pd.Series(target_count).sort_values().iloc[-50:].index

In [ ]:
# total_count

In [ ]:
# tokens = df_test['tokens'].iloc[0]

In [ ]:
# pd.Series(target_count.values()).value_counts()

In [ ]:
def predict(tokens):
    score = 0
    for token in tokens:
        p += p*(target_count[token])/(total_count[token]+0.0001)
    return p

In [ ]:
df_train['score'] = df_train['tokens'].apply(predict)

In [ ]:
df_test['score'] = df_test['tokens'].apply(predict)

In [ ]:
# df_test.sort_values('score').iloc[-100:]

In [ ]:
# df_test.sort_values('score')

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(df_train[target], df_train['score'])

In [ ]:
df_test['score'].max()

In [ ]:
vc = df[df[target]==1]['tokens'].apply(len).value_counts()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.bar(vc.index, vc.values)
plt.show()

In [ ]:
# рабоатет даже, если нет слова посоветуйте

In [ ]:
df_test[df_test['tokens'].apply(len)>3].sort_values('score').iloc[-50:]

In [ ]:
# топ признаков того, что это вопрос/просьба подсказать
stat = {}
for key, val in target_count.items():
    if val>=3:
        rate = (val)/(total_count[key]+1)
        stat[key] = rate

In [ ]:
stat['smm']

In [ ]:
# pd.Series(target_count).sort_values().iloc[-100:-50]

In [ ]:
pd.Series(stat).sort_values().iloc[-50:]